# Object detection

In [1]:
import os

from pathlib import Path

from IPython.display import display, Video

from video_intelligence.utils import io
from video_intelligence.config import Config
from video_intelligence.processing.detections import VideoProcessor
from video_intelligence.export.bq import BigQueryExporter

In [ ]:
config_file = str(Path.cwd().parent / "config.yaml")
config = Config.from_yaml(config_file)

In [3]:
gcs_io = io.GcsIO(config.project_id, f"gs://{config.bucket}")
local_io = io.LocalIO(root_path=config.paths.tmp)

In [4]:
videos = gcs_io.list_files(path=config.preprocessed_path, rec=True)
videos

['preprocessed/0000f77c-6257be58/0000f77c-6257be58.mp4',
 'preprocessed/00091078-7cff8ea6/00091078-7cff8ea6.mp4',
 'preprocessed/00225f53-67614580/00225f53-67614580.mp4']

In [5]:
test_videos = []
for _, gcs_path in enumerate(videos):
    video = gcs_io.get_video(path=gcs_path)
    path = gcs_path.split('/')[-1]
    local_io.save_video(video, path)
    test_videos.append(os.path.join(config.paths.tmp, path))

In [6]:
processor = VideoProcessor(config=config)

In [ ]:
results = {}

for idx, local_path in enumerate(test_videos):
    output_path, timestamps = processor.process_video(video_path=local_path)
    results[videos[idx]] = timestamps
    print(f"Processed video: {videos[idx]}")

    with open(output_path, "rb") as f:
        content = f.read()

    gcs_io.save_video(content, videos[idx].replace(
        config.preprocessed_path, config.output_path)
    )
    if output_path:
        display(Video(output_path, embed=True))

Video capture initialized.
Video properties: FPS=1.0, Width=1280, Height=480, Frames=10, Rotation=0


Processing Frames:   0%|          | 0/10 [00:00<?, ?frame/s]

In [ ]:
results

{'preprocessed/0000f77c-6257be58/0000f77c-6257be58.mp4': [Timestamp(timestamp='00:00', objects=[Detection(box_2d=[250, 478, 896, 761], label='car', confidence=0.98), Detection(box_2d=[346, 883, 855, 1000], label='car', confidence=0.92), Detection(box_2d=[306, 828, 367, 868], label='traffic sign', confidence=0.85), Detection(box_2d=[303, 881, 362, 916], label='traffic sign', confidence=0.82), Detection(box_2d=[228, 789, 303, 834], label='traffic sign', confidence=0.78), Detection(box_2d=[367, 828, 398, 868], label='traffic sign', confidence=0.75), Detection(box_2d=[362, 927, 410, 941], label='traffic light', confidence=0.7)]),
  Timestamp(timestamp='00:01', objects=[Detection(box_2d=[216, 433, 981, 784], label='car', confidence=0.95), Detection(box_2d=[245, 796, 891, 1000], label='car', confidence=0.92), Detection(box_2d=[299, 842, 353, 886], label='traffic sign', confidence=0.9), Detection(box_2d=[260, 409, 393, 512], label='traffic sign', confidence=0.88), Detection(box_2d=[438, 956, 

In [ ]:
import json
with open(config.paths.results, 'w', encoding='utf-8') as f:
    serializable_results = {
        video_path: [timestamp.to_dict() for timestamp in timestamps]
        for video_path, timestamps in results.items()
    }
    json.dump(serializable_results, f, indent=4)

In [ ]:
exporter = BigQueryExporter(config)
exporter.export_detections(config.paths.results)